In [2]:
import pandas as pd
from pathlib import Path

# Loading in Data

Since we're making the model for Roger Federer, let's load in his data from `feature-engineering.ipynb`.

In [3]:
data_file = Path('./data', 'fed_2018.hdf')
fed_2018 = pd.read_hdf(data_file, 'fed_2018')
fed_2018.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,...,l_1stWon%,w_2ndWon%,l_2ndWon%,w_bpSaved%,l_bpSaved%,w_bpFaced%,l_bpFaced%,result,win_streak,head_to_head
202,2018-580,Australian Open,Hard,128,G,2018-01-15,164,103819,Roger Federer,R,...,0.700000,0.608696,0.450000,1.000000,0.692308,0.024691,0.144444,1,1,1.0
234,2018-580,Australian Open,Hard,128,G,2018-01-15,232,103819,Roger Federer,R,...,0.759259,0.666667,0.509434,0.666667,0.727273,0.032609,0.102804,1,2,1.0
250,2018-580,Australian Open,Hard,128,G,2018-01-15,316,103819,Roger Federer,R,...,0.672131,0.424242,0.413793,0.500000,0.375000,0.021277,0.088889,1,3,1.0
258,2018-580,Australian Open,Hard,128,G,2018-01-15,408,103819,Roger Federer,R,...,0.666667,0.757576,0.529412,NaN,0.700000,0.000000,0.092593,1,4,1.0
262,2018-580,Australian Open,Hard,128,G,2018-01-15,504,103819,Roger Federer,R,...,0.698113,0.527778,0.512195,0.600000,0.500000,0.052083,0.085106,1,5,1.0
